# Preprocess Pipeline Demo

This notebook demonstrates the basic preprocessing functions in `ml/data/preprocess.py`:

1. **`to_epoch_seconds`** - Convert datetime to epoch seconds (since 2025-01-01)
2. **`add_closest_points`** - Find closest point on route polylines for GPS coordinates

These functions are used by `preprocess_pipeline()` in `ml/pipelines.py`.

In [2]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
from datetime import datetime, timezone

# Add project root to Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import preprocessing functions
from ml.data.preprocess import to_epoch_seconds, add_closest_points, EPOCH_2025_OFFSET

print(f"EPOCH_2025_OFFSET: {EPOCH_2025_OFFSET}")
print(f"This corresponds to: {datetime.fromtimestamp(EPOCH_2025_OFFSET, tz=timezone.utc)}")

EPOCH_2025_OFFSET: 1735689600
This corresponds to: 2025-01-01 00:00:00+00:00


---
## 1. `to_epoch_seconds`

Converts a datetime column to epoch seconds since 2025-01-01 00:00:00 UTC.

**Why?** Smaller numbers are easier for ML models to work with.

### Function Signature
```python
def to_epoch_seconds(
    df: pd.DataFrame,
    input_column: str,
    output_column: str
) -> None:
```

**Modifies DataFrame in-place.**

In [3]:
# Create sample DataFrame with timestamps
df_epoch = pd.DataFrame({
    'vehicle_id': [1, 1, 1, 2, 2],
    'timestamp': pd.to_datetime([
        '2025-01-01 00:00:00',  # Exactly at epoch start
        '2025-01-01 00:01:00',  # 60 seconds later
        '2025-01-02 12:00:00',  # 1.5 days later
        '2025-06-15 08:30:00',  # Mid-year
        '2025-12-31 23:59:59'   # End of year
    ], utc=True),
    'latitude': [42.7284, 42.7295, 42.7300, 42.7310, 42.7320],
    'longitude': [-73.6788, -73.6799, -73.6810, -73.6820, -73.6830]
})

print("BEFORE to_epoch_seconds:")
print(df_epoch)
print(f"\nColumns: {list(df_epoch.columns)}")

BEFORE to_epoch_seconds:
   vehicle_id                 timestamp  latitude  longitude
0           1 2025-01-01 00:00:00+00:00   42.7284   -73.6788
1           1 2025-01-01 00:01:00+00:00   42.7295   -73.6799
2           1 2025-01-02 12:00:00+00:00   42.7300   -73.6810
3           2 2025-06-15 08:30:00+00:00   42.7310   -73.6820
4           2 2025-12-31 23:59:59+00:00   42.7320   -73.6830

Columns: ['vehicle_id', 'timestamp', 'latitude', 'longitude']


In [4]:
# Apply to_epoch_seconds
to_epoch_seconds(df_epoch, 'timestamp', 'epoch_seconds')

print("AFTER to_epoch_seconds:")
print(df_epoch)
print(f"\nNew column added: 'epoch_seconds'")
print(f"\nEpoch seconds values:")
for i, row in df_epoch.iterrows():
    print(f"  {row['timestamp']} → {row['epoch_seconds']:.0f} seconds since 2025-01-01")

AFTER to_epoch_seconds:
   vehicle_id                 timestamp  latitude  longitude  epoch_seconds
0           1 2025-01-01 00:00:00+00:00   42.7284   -73.6788            0.0
1           1 2025-01-01 00:01:00+00:00   42.7295   -73.6799           60.0
2           1 2025-01-02 12:00:00+00:00   42.7300   -73.6810       129600.0
3           2 2025-06-15 08:30:00+00:00   42.7310   -73.6820     14286600.0
4           2 2025-12-31 23:59:59+00:00   42.7320   -73.6830     31535999.0

New column added: 'epoch_seconds'

Epoch seconds values:
  2025-01-01 00:00:00+00:00 → 0 seconds since 2025-01-01
  2025-01-01 00:01:00+00:00 → 60 seconds since 2025-01-01
  2025-01-02 12:00:00+00:00 → 129600 seconds since 2025-01-01
  2025-06-15 08:30:00+00:00 → 14286600 seconds since 2025-01-01
  2025-12-31 23:59:59+00:00 → 31535999 seconds since 2025-01-01


In [5]:
# Verify the conversion
print("Verification:")
print(f"  Row 0 (2025-01-01 00:00:00): epoch_seconds = {df_epoch.loc[0, 'epoch_seconds']:.0f} (expected: 0)")
print(f"  Row 1 (2025-01-01 00:01:00): epoch_seconds = {df_epoch.loc[1, 'epoch_seconds']:.0f} (expected: 60)")
print(f"  Row 2 (2025-01-02 12:00:00): epoch_seconds = {df_epoch.loc[2, 'epoch_seconds']:.0f} (expected: {24*3600 + 12*3600})")

Verification:
  Row 0 (2025-01-01 00:00:00): epoch_seconds = 0 (expected: 0)
  Row 1 (2025-01-01 00:01:00): epoch_seconds = 60 (expected: 60)
  Row 2 (2025-01-02 12:00:00): epoch_seconds = 129600 (expected: 129600)


---
## 2. `add_closest_points`

For each GPS coordinate, finds the closest point on any route polyline.

**Returns (via output_columns mapping):**
- `distance`: Distance to closest point (km)
- `closest_point_lat`: Latitude of closest point
- `closest_point_lon`: Longitude of closest point
- `route_name`: Name of the closest route
- `polyline_index`: Index of polyline within route
- `segment_index`: Index of segment within polyline

### Function Signature
```python
def add_closest_points(
    df: pd.DataFrame,
    lat_column: str,
    lon_column: str,
    output_columns: dict[str, str],
    additive: bool = False
) -> None:
```

**Modifies DataFrame in-place.**

In [6]:
# Create sample DataFrame with RPI campus coordinates
# These are real coordinates near shuttle routes
df_closest = pd.DataFrame({
    'vehicle_id': [1, 1, 1, 2, 2],
    'latitude': [
        42.7302,   # Near Student Union (WEST route)
        42.7284,   # Near 15th St (WEST route)
        42.7318,   # Near Polytechnic (NORTH route)
        42.7335,   # Near West Hall (WEST route)
        42.7260    # Near Blitman (both routes)
    ],
    'longitude': [
        -73.6762,
        -73.6788,
        -73.6720,
        -73.6860,
        -73.6780
    ]
})

print("BEFORE add_closest_points:")
print(df_closest)
print(f"\nColumns: {list(df_closest.columns)}")

BEFORE add_closest_points:
   vehicle_id  latitude  longitude
0           1   42.7302   -73.6762
1           1   42.7284   -73.6788
2           1   42.7318   -73.6720
3           2   42.7335   -73.6860
4           2   42.7260   -73.6780

Columns: ['vehicle_id', 'latitude', 'longitude']


In [7]:
# Apply add_closest_points with all output columns
add_closest_points(df_closest, 'latitude', 'longitude', {
    'distance': 'dist_to_route',
    'route_name': 'route',
    'closest_point_lat': 'closest_lat',
    'closest_point_lon': 'closest_lon',
    'polyline_index': 'polyline_idx',
    'segment_index': 'segment_idx'
})

print("AFTER add_closest_points:")
print(df_closest)
print(f"\nNew columns added: {['dist_to_route', 'route', 'closest_lat', 'closest_lon', 'polyline_idx', 'segment_idx']}")

100%|██████████| 5/5 [00:00<00:00, 693.85it/s]

Done! Adding columns to dataframe.
AFTER add_closest_points:
   vehicle_id  latitude  longitude  dist_to_route  route  closest_lat  \
0           1   42.7302   -73.6762            NaN    NaN          NaN   
1           1   42.7284   -73.6788            NaN    NaN          NaN   
2           1   42.7318   -73.6720       0.011565  NORTH    42.731763   
3           2   42.7335   -73.6860            NaN    NaN          NaN   
4           2   42.7260   -73.6780       0.028192   WEST    42.726108   

   closest_lon  polyline_idx  segment_idx  
0          NaN           NaN          NaN  
1          NaN           NaN          NaN  
2   -73.671868           0.0         23.0  
3          NaN           NaN          NaN  
4   -73.678312           1.0          7.0  

New columns added: ['dist_to_route', 'route', 'closest_lat', 'closest_lon', 'polyline_idx', 'segment_idx']


In [8]:
# Detailed view of each point's route matching
print("Route Matching Results:")
print("="*70)
for i, row in df_closest.iterrows():
    print(f"\nPoint {i}:")
    print(f"  Original: ({row['latitude']:.4f}, {row['longitude']:.4f})")
    print(f"  Closest:  ({row['closest_lat']:.4f}, {row['closest_lon']:.4f})")
    print(f"  Route: {row['route']} (polyline {row['polyline_idx']}, segment {row['segment_idx']})")
    print(f"  Distance to route: {row['dist_to_route']*1000:.1f} meters")

Route Matching Results:

Point 0:
  Original: (42.7302, -73.6762)
  Closest:  (nan, nan)
  Route: nan (polyline nan, segment nan)
  Distance to route: nan meters

Point 1:
  Original: (42.7284, -73.6788)
  Closest:  (nan, nan)
  Route: nan (polyline nan, segment nan)
  Distance to route: nan meters

Point 2:
  Original: (42.7318, -73.6720)
  Closest:  (42.7318, -73.6719)
  Route: NORTH (polyline 0.0, segment 23.0)
  Distance to route: 11.6 meters

Point 3:
  Original: (42.7335, -73.6860)
  Closest:  (nan, nan)
  Route: nan (polyline nan, segment nan)
  Distance to route: nan meters

Point 4:
  Original: (42.7260, -73.6780)
  Closest:  (42.7261, -73.6783)
  Route: WEST (polyline 1.0, segment 7.0)
  Distance to route: 28.2 meters


### Additive Mode

When `additive=True`, only processes rows where output columns have NaN values.
Useful for incremental updates.

In [9]:
# Create DataFrame with some rows already processed
df_additive = pd.DataFrame({
    'vehicle_id': [1, 1, 1],
    'latitude': [42.7302, 42.7284, 42.7318],
    'longitude': [-73.6762, -73.6788, -73.6720],
    'route': ['WEST', None, None],  # Only first row has route
    'dist_to_route': [0.001, np.nan, np.nan]
})

print("BEFORE add_closest_points (additive=True):")
print(df_additive)

# Apply in additive mode - should only process rows 1 and 2
add_closest_points(df_additive, 'latitude', 'longitude', {
    'route_name': 'route',
    'distance': 'dist_to_route'
}, additive=True)

print("\nAFTER add_closest_points (additive=True):")
print(df_additive)
print("\n→ Row 0 was preserved, rows 1-2 were filled in")

BEFORE add_closest_points (additive=True):
   vehicle_id  latitude  longitude route  dist_to_route
0           1   42.7302   -73.6762  WEST          0.001
1           1   42.7284   -73.6788  None            NaN
2           1   42.7318   -73.6720  None            NaN
Additive mode: Processing 2/3 rows with missing closest points


100%|██████████| 2/2 [00:00<00:00, 1006.55it/s]

Done! Adding columns to dataframe.

AFTER add_closest_points (additive=True):
   vehicle_id  latitude  longitude  route  dist_to_route
0           1   42.7302   -73.6762   WEST       0.001000
1           1   42.7284   -73.6788   None            NaN
2           1   42.7318   -73.6720  NORTH       0.011565

→ Row 0 was preserved, rows 1-2 were filled in


---
## Integration with `preprocess_pipeline`

In `ml/pipelines.py`, these functions are used together:

In [10]:
# Show how preprocess_pipeline uses these functions
print("""
def preprocess_pipeline(df: pd.DataFrame = None, **kwargs) -> pd.DataFrame:
    from ml.data.preprocess import to_epoch_seconds, add_closest_points

    # ... load data if df is None ...

    # Step 1: Add epoch seconds
    to_epoch_seconds(df, 'timestamp', 'epoch_seconds')

    # Step 2: Add route information
    add_closest_points(df, 'latitude', 'longitude', {
        'distance': 'dist_to_route',
        'route_name': 'route',
        'closest_point_lat': 'closest_lat',
        'closest_point_lon': 'closest_lon',
        'polyline_index': 'polyline_idx',
        'segment_index': 'segment_idx'
    })

    return df
""")


def preprocess_pipeline(df: pd.DataFrame = None, **kwargs) -> pd.DataFrame:
    from ml.data.preprocess import to_epoch_seconds, add_closest_points

    # ... load data if df is None ...

    # Step 1: Add epoch seconds
    to_epoch_seconds(df, 'timestamp', 'epoch_seconds')

    # Step 2: Add route information
    add_closest_points(df, 'latitude', 'longitude', {
        'distance': 'dist_to_route',
        'route_name': 'route',
        'closest_point_lat': 'closest_lat',
        'closest_point_lon': 'closest_lon',
        'polyline_index': 'polyline_idx',
        'segment_index': 'segment_idx'
    })

    return df



In [11]:
# Run the actual preprocess_pipeline on real data
from ml.pipelines import preprocess_pipeline

# Load preprocessed data (uses cache if available)
df = preprocess_pipeline()

print(f"Preprocessed {len(df):,} records")
print(f"\nColumns: {list(df.columns)}")
print(f"\nSample data:")
df[['vehicle_id', 'timestamp', 'epoch_seconds', 'latitude', 'longitude', 'route', 'dist_to_route']].head(10)

2026-02-04 17:17:50 - ml.cache - INFO - Loading preprocessed data from /Users/joel/eclipse-workspace/shuttletracker-new/ml/cache/shared/locations_preprocessed.csv
2026-02-04 17:17:54 - ml.cache - INFO - Loaded 1833872 records from cache
Preprocessed 1,833,872 records

Columns: ['vehicle_id', 'latitude', 'longitude', 'timestamp', 'epoch_seconds', 'dist_to_route', 'route', 'closest_lat', 'closest_lon', 'polyline_idx', 'segment_idx']

Sample data:


,vehicle_id,timestamp,epoch_seconds,latitude,longitude,route,dist_to_route
0,281474977379809,2025-07-31 15:53:49.141,1.828763e+07,42.730284,-73.676562,NaN,NaN
1,281474977371235,2025-07-31 15:55:27.052,1.828773e+07,42.723882,-73.680926,WEST,0.001679
2,281474977371235,2025-07-31 15:55:33.027,1.828773e+07,42.724508,-73.681213,WEST,0.001747
3,281474977371235,2025-07-31 15:55:44.248,1.828774e+07,42.725621,-73.681507,WEST,0.000072
4,281474977371235,2025-07-31 15:55:50.028,1.828775e+07,42.726247,-73.681568,WEST,0.000630
5,281474977371235,2025-07-31 15:55:55.999,1.828776e+07,42.726762,-73.681983,WEST,0.003545
6,281474977371235,2025-07-31 15:56:01.021,1.828776e+07,42.727014,-73.682665,WEST,0.005010
7,281474977371235,2025-07-31 15:56:06.019,1.828777e+07,42.727261,-73.683403,WEST,0.004937
8,281474977371235,2025-07-31 15:56:11.521,1.828777e+07,42.727473,-73.684111,WEST,0.003928
9,281474977371235,2025-07-31 15:56:17.045,1.828778e+07,42.727602,-73.684729,WEST,0.003714


---
## Summary

| Function | Input | Output | Purpose |
|----------|-------|--------|----------|
| `to_epoch_seconds` | datetime column | epoch seconds column | ML-friendly timestamps |
| `add_closest_points` | lat/lon columns | route, distance, closest point | Route matching |

Both functions modify the DataFrame **in-place**.